# Reading the data

In [2]:
#@title Imports { form-width: "150px" }
import networkx as nx
from random import choice
import numpy as np
import pandas as pd
import requests
import zipfile
import time
import pickle
import random

from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans, Birch
# from sklearn.datasets.samples_generator import make_blobs
import hashlib
import scipy

import itertools

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.model_selection import GridSearchCV


from sklearn.cross_decomposition import PLSRegression
from sklearn.cross_decomposition import PLSCanonical

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.neural_network import MLPRegressor

from sklearn.neighbors import KNeighborsRegressor

# For the Kendall’s tau correlation
import scipy.stats as stats

In [3]:
#@title read_data_functions { form-width: "150px" }
def read_txt_data(address, sep_in = ' '):
  """
  Returns a graph with node indices that start from zero.
  """
  with open(address, "r") as file:
    lines = file.readlines()

  edges = []
  for line in lines:
    stripped_line = line.strip().split(sep= sep_in)
    # print(stripped_line)
    try:
      edges.append((int(stripped_line[0]), int(stripped_line[1])))
    except:
      # print(stripped_line)
      # print(stripped_line[0].split())
      if len(stripped_line) == 1:
        double_split = stripped_line[0].split()
        edges.append((int(double_split[0]), int(double_split[1])))
        # print(f'{(int(double_split[0]), int(double_split[1]))} appended')
      elif len(stripped_line) > 1:
        edges.append((int(stripped_line[0]), int(stripped_line[1])))
        # print(f'{edges.append((int(stripped_line[0]), int(stripped_line[1])))} appended')
  
  G = nx.from_edgelist(edges)
  GX = nx.to_numpy_matrix(G)
  G = nx.from_numpy_matrix(GX)

  return G


def read_gml_data(address):
  """
  Returns a graph with node indices that start from zero.
  """
  G = nx.read_gml(address)
  GX = nx.to_numpy_matrix(G)
  G = nx.from_numpy_matrix(GX)

  return G

In [4]:
#@title load networks in G1 to G8 { form-width: "150px" }
graphs = []

### Loading the euroroad network from http://networkrepository.com/inf-euroroad.php
G5 = read_txt_data("/content/drive/My Drive/PhD_IM/datasets/G5_euroroad.txt")
graphs.append(G5)

### Loading Hamsterster network from http://networkrepository.com/soc-hamsterster.php
G6 = read_txt_data("/content/drive/My Drive/PhD_IM/datasets/G6_soc_hamsterster.txt")
graphs.append(G6)

### Loading powergrid network from http://konect.cc/networks/opsahl-powergrid/
G7 = read_txt_data("/content/drive/My Drive/PhD_IM/datasets/G7_powergrid_konnekt.txt")
graphs.append(G7)

### Loading pgp network from http://networkrepository.com/tech-pgp.php
G8 = read_txt_data("/content/drive/My Drive/PhD_IM/datasets/G8_pgp_repos.txt")
graphs.append(G8)

### Loading the euroroad network from http://networkrepository.com/inf-euroroad.php
G9 = read_txt_data("/content/drive/MyDrive/PhD_IM/datasets/G9_email.txt")
graphs.append(G9)

### Loading powergrid network from http://konect.cc/networks/opsahl-powergrid/
G10 = read_txt_data("/content/drive/MyDrive/PhD_IM/datasets/G10_citeseer.txt", ',')
graphs.append(G10)

### Loading Hamsterster network from http://networkrepository.com/soc-hamsterster.php
G11 = read_txt_data("/content/drive/MyDrive/PhD_IM/datasets/G11_collaborations.txt")
graphs.append(G11)

G12 = read_txt_data("/content/drive/MyDrive/PhD_IM/datasets/G12_Delaunay.txt")
graphs.append(G12)

G13 = read_txt_data("/content/drive/MyDrive/PhD_IM/datasets/G9_Erdos.txt")
graphs.append(G13)


In [5]:
G = G11
print(f'n_nodes: {len(G.nodes)}, n_edges:{len(G.edges)}, max_degree:{np.max([G.degree(v) for v in G.nodes])}, mean_deg:{np.mean([G.degree(v) for v in G.nodes])}')

n_nodes: 4158, n_edges:13422, max_degree:81, mean_deg:6.455988455988456


In [6]:
#@title get_network_df_2 { form-width: "50px" }
def get_network_df(in_saving = False):

  g_names=["G5"     ,       "G6",          "G7"      , "G8"    ,  "G9"        ,  "G10"     ,    "G11", "G12"       , "G13"] 
  names = ["euroroad", "hamster",    "powergrid"     , "pgp"   ,   "email"    , "cietseer", "collab",  "delaunay", 'erdos']

  thresh_sir = [0.35    ,  0.03     ,  0.3           ,  0.1     , 0.1        ,  0.3        ,  0.05  ,  0.1      ,  0.08]
  range_sir = [0.15     ,   0.01    ,   0.15         ,   0.15    ,  0.05       ,   0.1   ,   0.015 , 0.15      , 0.015]
  step_sir =  [0.01     ,   0.001   ,   0.01         ,   0.01    ,  0.001      ,    0.01    ,  0.001 , 0.01      , 0.001]
  size_sir   = [1      ,   1       ,   1             ,   1     ,  1           ,  1         ,    1    ,   1       ,   1]


  # thresh_tip = [0.2 ,  0.2  ,  0.2      ,   0.2           ,  0.2   ,  0.2     ,  0.2]
  # # thresh_tip = [0.4 ,  0.4  ,  0.4      ,   0.4           ,  0.4   ,  0.2     ,  0.2]
  # # thresh_tip = [0.6 ,  0.6  ,  0.6      ,   0.6           ,  0.6   ,  0.2     ,  0.2]
  # size_tip   = [1    ,   1   ,   1        ,    1           ,   1     ,   1      ,   0.3]

  sir_gt = []
  sir_indi = []

  for i in range(len(g_names)):
    sir_root = '/content/drive/MyDrive/PhD_IM/datasets/gt_data'
    folder_name = f'{g_names[i]}_{names[i]}'
    
    file_name_sir = f'{g_names[i]}_{thresh_sir[i]}_gt.pckl'
    sir_gt.append(pickle.load(open(f'{sir_root}/{folder_name}/{file_name_sir}', 'rb')))


    file_name_sir_indi = f'{g_names[i]}_{thresh_sir[i]}_{size_sir[i]}_indices.pckl'
    if size_sir[i] < 1:
      try:
        sir_indi.append(pickle.load(open(f'{sir_root}/{folder_name}/{file_name_sir_indi}', 'rb')))
      except:
        sir_indi.append(graphs[i].nodes)
    else:
      sir_indi.append(graphs[i].nodes)

  
  # tip_gt = []
  # tip_indi = []

  # for i in range(len(g_names)):
  #   tip_root = '/content/drive/My Drive/PhD_IM/datasets/gt_data_TIP'
  #   folder_name = f'{g_names[i]}_{names[i]}'
    
  #   file_name_tip = f'{g_names[i]}_{thresh_tip[i]}_{size_tip[i]}_gt_tip.pckl'
  #   tip_gt.append(pickle.load(open(f'{tip_root}/{folder_name}/{file_name_tip}', 'rb')))

  #   file_name_tip_indi = f'{g_names[i]}_{thresh_tip[i]}_{size_tip[i]}_indices_tip.pckl'
  #   if size_tip[i] < 1:
  #     tip_indi.append(pickle.load(open(f'{tip_root}/{folder_name}/{file_name_tip_indi}', 'rb')))  
  #   else:
  #     tip_indi.append(graphs[i].nodes)



  df_net = pd.DataFrame(index=g_names,
                        columns=['name',  'graph', 'thresh_sir', 'thresh_tip',
                                 'SIR_gt', 'SIR_indi', 'TIP_gt', 'TIP_indi'])
  df_net['name'] = names
  df_net['graph'] = graphs
  df_net['thresh_sir'] = thresh_sir
  df_net['range_sir'] = range_sir
  df_net['step_sir'] = step_sir
  # df_net['thresh_tip'] = thresh_tip
  df_net['size_sir']  = size_sir
  # df_net['size_tip']  = size_tip
  df_net['SIR_gt'] = sir_gt
  df_net['SIR_indi'] = sir_indi
  # df_net['TIP_gt'] = tip_gt
  # df_net['TIP_indi'] = tip_indi

  return df_net

# Evaluation Functions

## The Monotonicity Relation

$M(R) = 1 - \frac{\sum_{r\in R} n_r \times (n_r - 1)}{n \times (n-1)}$

$n_r$: the number of elements with rank $r$ in ranking list $R$.

$n$: the number of elements with distinct rank.

$M(R)$ would have a value between $0$ and $1$. Higher values show greater
differentiation and uniformity for ranking list $R$.

In [7]:
#@title get_monotinicity_relation
def get_monotinicity_relation(R):
  '''
  R: the ranking array with the rank of elements at each index instead of value.
  '''
  # n is the total number of nodes 
  n = len(R)

  n_r =  np.array([sum(R == i) for i in range(n)])

  M_R = 1 - (sum([ n_r[i] * (n_r[i] - 1) for i in range(n)]) / (n * (n-1)))

  return M_R ** 2

## Get Ranking

In [8]:
#@title get_ranking { form-width: "150px" }
def get_ranking(input_array):
  '''
  input_array: An array of different values.
  ranking: for each index, returns a number showing the rank of item at that index

  Currently, we assume that elements with same value would have the same rank.
  '''
  unique_input = list(set(input_array))
  sorted_unique = sorted(unique_input, reverse=True)
  
  ranking = np.empty_like(input_array, dtype=np.int64) 

  for rank, value in enumerate(sorted_unique):
    ranking[input_array == value] = rank

  return ranking

In [9]:
#@title get_sorted_ranking { form-width: "150px" }
def get_sorted_ranking(ranking):
  input_ranking = zip(ranking, np.arange(len(ranking)))
  return [int(val) for rank, val in sorted(input_ranking)]

In [10]:
#@title optimalK { form-width: "150px" }
def optimalK(data, nrefs=6, maxClusters=18):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal
    

In [11]:
#@title elbow_optimal { form-width: "150px" }
def elbow_optimal(data, metric, k = 30):
  model = KElbowVisualizer(KMeans(), k=k, metric=metric)
  model.fit(data)
  return model.elbow_value_

In [12]:
#@title get_cluster_sampling { form-width: "150px" }

def get_cluster_sampling(G, samp_size, size_method, elbow_metric):
  '''
  size_method: can be either 'elbow' or 'gap'
  elbow_method: can be either 'distortion', 'silhouette', 'calinski_harabasz'
  '''

  n_samples = int(np.ceil(samp_size * len(G.nodes)))
  B = get_features(G)

  if size_method == 'gap':
    optimal_clus_number = optimalK(B, nrefs=6, maxClusters=18)[0]
  elif size_method == 'elbow':
    optimal_clus_number = elbow_optimal(B, elbow_metric)
  else:
    print('Invalid size_method')
    raise Exception

  sample_per_cluster = int(np.ceil(n_samples / optimal_clus_number))
  # out_of_bag = n_samples - sample_per_cluster * optimal_clus_number

  # if out_of_bag >= optimal_clus_number:
  #   sample_per_cluster += 1
  #   out_of_bag -= optimal_clus_number

  kmeans_model = KMeans(n_clusters=optimal_clus_number).fit(B)
  B_labels = kmeans_model.labels_
  B_indices = np.array([i for i in range(len(G.nodes))])

  labels = sorted(set(B_labels))

  elem_per_clus = [len(B_indices[B_labels == label]) for label in labels]
  sample_per_clus = [sample_per_cluster if elem >= sample_per_cluster else elem \
                     for elem in elem_per_clus]
  out_of_bag = n_samples - np.sum(sample_per_clus)
  print(out_of_bag)
  print(n_samples)
  print(sample_per_cluster)
  print(np.sum(sample_per_clus))
  # raise Exception

  
  cur = 0
  len_cluss = len(sample_per_clus)
  while out_of_bag >= 1:
    if elem_per_clus[cur] > sample_per_clus[cur]:
      sample_per_clus[cur] += 1
      out_of_bag -= 1

    cur += 1
    cur = cur % len_cluss

  sampled_nodes = []

  for label in labels:
    clust = list(B_indices[B_labels == label])
    sel_indices = list(random.sample(clust, sample_per_clus[label]))
    sampled_nodes.extend(sel_indices)

  sampled_nodes = sorted(sampled_nodes)

  # make sure that the number of output is equal to samp_size
  print(len(sampled_nodes))
  print(n_samples)
  print(out_of_bag)
  assert len(sampled_nodes) == n_samples

  # if len(sampled_nodes) != n_samples:
  #   print(len(sampled_nodes))
  #   print(n_samples)
  #   raise Exception

  return sampled_nodes

# The model

## Other models

In [13]:
#@title ks { form-width: "110px" }
def ks(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  cores = np.zeros_like(np.array(G.nodes))
  coreness = np.min([deg for v, deg in G.degree])

  while len(G.nodes):
    core_nodes = []
    for node, deg in G.degree:
      if deg == coreness:
        cores[node] = coreness
        core_nodes.append(node)
    
    G.remove_nodes_from(core_nodes)
    
    sub_core_nodes = []
    for node, deg in G.degree:
      if deg <= coreness:
        cores[node] = coreness
        sub_core_nodes.append(node)
    
    G.remove_nodes_from(sub_core_nodes)

    coreness += 1
  
  return cores

In [14]:
#@title ks-if-2 { form-width: "110px" }
def ks_if_2(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  original = nx.Graph()
  original.add_nodes_from(np.array(G_in.nodes))
  original.add_edges_from(np.array(G_in.edges))

  cores = np.zeros_like(np.array(G.nodes))
  iterations = np.zeros_like(np.array(G.nodes))

  coreness = np.min([deg for v, deg in G.degree])

  while len(G.nodes):
    core_nodes = []
    for node, deg in G.degree:
      if deg == coreness:
        cores[node] = coreness * 1.5
        iterations[node] = 1
        core_nodes.append(node)
    
    G.remove_nodes_from(core_nodes)
    
    sub_core_nodes = []
    for node, deg in G.degree:
      if deg <= coreness:
        cores[node] = coreness * 2
        iterations[node] = 2
        sub_core_nodes.append(node)
    
    G.remove_nodes_from(sub_core_nodes)

    coreness += 1

    return cores, iterations

In [15]:
#@title iks
def iks(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  cores = np.zeros_like(np.array(G.nodes))
  coreness = np.min([deg for v, deg in G.degree])

  while len(G.nodes):
    core_nodes = []
    degs = np.array([deg for v, deg in G.degree])
    min_deg = np.min(degs)

    for node, deg in G.degree:
      if deg == min_deg:
        cores[node] = coreness
        core_nodes.append(node)

    G.remove_nodes_from(core_nodes)

    coreness += 1
  
  return cores

In [16]:
#@title ks-if { form-width: "110px" }
def ks_if(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  original = nx.Graph()
  original.add_nodes_from(np.array(G_in.nodes))
  original.add_edges_from(np.array(G_in.edges))

  cores = np.zeros_like(np.array(G.nodes))
  iterations = np.zeros_like(np.array(G.nodes))

  coreness = np.min([deg for v, deg in G.degree])

  while len(G.nodes):
    core_nodes = []
    for node, deg in G.degree:
      if deg == coreness:
        cores[node] = coreness * 1.5
        iterations[node] = 1
        core_nodes.append(node)
    
    G.remove_nodes_from(core_nodes)
    
    sub_core_nodes = []
    for node, deg in G.degree:
      if deg <= coreness:
        cores[node] = coreness * 2
        iterations[node] = 2
        sub_core_nodes.append(node)
    
    G.remove_nodes_from(sub_core_nodes)

    coreness += 1
  
  degs = np.array([deg for v, deg in original.degree])
  ksif_score = np.zeros_like(np.array(G_in.nodes))

  for node in original.nodes:
    ksif_neighbs = np.sum([cores[u] * degs[u] for u in original.neighbors(node)])
    ksif_score[node] = ksif_neighbs + cores[node] * degs[node] * 1/4


  return ksif_score

In [17]:
#@title mmd { form-width: "110px" }
def mmd(G_in, landa = 0.7):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  'k_m = deg_existing + \lambda * deg_removed'
  'k_m = k_r + \lambda * k_e'
  cores = np.zeros_like(np.array(G.nodes))
  nodes = np.array(G.nodes)
  G_e = nx.empty_graph(n=len(G.nodes))

  G_r = nx.empty_graph(n=len(G.nodes))
  G_r.add_edges_from(np.array(G.edges))

  remained = np.array([True] * len(G.nodes))


  while len(G.nodes):
    k_r = np.array([deg for v, deg in G_r.degree])
    k_e = np.array([deg for v, deg in G_e.degree])
    k_m = k_r + landa * k_e

    k_m_new = k_m[remained]

    mmd_score = np.min(k_m_new)
    new_core = nodes[k_m == mmd_score]

    remained[k_m == mmd_score] = False

    cores[k_m == mmd_score] = mmd_score

    core_edges = G.edges(new_core)
    G_e.add_edges_from(core_edges)
    G_r.remove_nodes_from(core_edges)
    
    G.remove_nodes_from(new_core)

  
  return cores



In [18]:
#@title cnc { form-width: "110px" }
def cnc(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  'score = sum corenes for all neighbours'
  ks_score = ks(G)
  cnc_score = np.zeros_like(np.array(G.nodes))

  for node in G.nodes:
    temp_cnc = 0
    for neighbour in G.neighbors(node):
      temp_cnc += ks_score[neighbour]
    cnc_score[node] = temp_cnc
  
  return cnc_score

In [19]:
#@title mcde { form-width: "110px" }
def mcde(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  ks_score = ks(G)

  A1 = nx.adjacency_matrix(G).astype(bool).toarray()
  mcde_score = np.zeros_like(G.nodes)
  entropyy = np.zeros_like(G.nodes, dtype=np.float32)

  degs = np.array([deg for v, deg in G.degree])

  for node in G.nodes:
    entropy_temp = 0

    for core_level in range(np.max(ks_score) + 1):
      if degs[node] > 0:
        p_i = np.sum((ks_score == core_level) & A1[node]) / degs[node]
        if p_i > 0 :
          entropy_temp +=  p_i * np.log(p_i)
      
    entropyy[node] = -entropy_temp

  mcde_score = entropyy + degs + ks_score

  return np.round(mcde_score, 3)
      


In [20]:
def entropy(X):
  return -np.sum([0 if x ==0 else x*np.log(x) for x in X])

In [21]:
#@title DSR { form-width: "110px" }
def DSR(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  'DSC(v) =  IKS_bar(v) + H_1(v) + JSD(u \in neighs(v))'
  'IKS_bar(v) = IKS(nighbs(v)) / |neighbs(v)|'
  iks_score = iks(G)
  degs = np.array([deg for v, deg in G.degree])
  IKS_neighbs = np.zeros_like(G.nodes)

  for node in G.nodes:
    iks_neighbs = 0
    for neighbour in G.neighbors(node):
      iks_neighbs += iks_score[neighbour]
    IKS_neighbs[node] = iks_neighbs
  
  IKS_bar = IKS_neighbs / degs

  'H_1(v) = Entropy (IKS(u) / IKS_neighbs(v))'
  H_1 = np.zeros_like(G.nodes)
  for v in G.nodes:
    for u in G.neighbors(node):
      H_1[v] += iks_score[u]/IKS_neighbs[v] * np.log(iks_score[u]/IKS_neighbs[v])
    H_1[v] = -H_1[v]
  'JSD(u \in neighbs(v) = entropy(sum(1/deg(v) * P(neighbs))) - 1/deg(v) * sum(entropy(P(neighbs)))'
  # First need to compute the probability of presence of neighbours of nodes in different shells.

  # computing the shell vectors
  A = nx.adjacency_matrix(G).astype(bool).toarray()
  P = np.zeros(shape=(len(G.nodes), np.max(iks_score) + 1))

  for node in G.nodes:
    for core in range(np.max(iks_score ) + 1):
      if np.sum(iks_score == core) == 0:
        P[node, core] = 0
      else:
        P[node, core] = np.sum((iks_score == core) & A[node])/ np.sum(iks_score == core)
  
  JSD = np.zeros_like(G.nodes, dtype=np.float32)
  for v in G.nodes:
    sum_p = 0
    second_term = 0
    for u in G.neighbors(v):
      sum_p += P[u]
      second_term += entropy(P[u])
    # print(sum_p)
    # print(second_term)
    first_term = entropy(sum_p / degs[v])
    second_term = second_term / degs[v]
    JSD[v] = first_term - second_term
  
  DSC_score = IKS_bar + H_1 + JSD
  DSR_score = np.zeros_like(G.nodes, dtype=np.float32)
  for node in G.nodes:
    dsr_temp = 0
    for neighb in G.neighbors(node):
      dsr_temp += DSC_score[neighb]
    DSR_score[node] = dsr_temp
  
  return np.round(DSR_score, 3)


In [22]:
#@title EDSR { form-width: "110px" }
def EDSR(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  DSR_score = DSR(G)
  EDSR_score = np.zeros_like(G.nodes)

  for node in G.nodes:
    edsr_temp = 0
    for neighb in G.neighbors(node):
      edsr_temp += DSR_score[neighb]
    EDSR_score[node] = edsr_temp
  
  return EDSR_score

In [23]:
#@title CRM { form-width: "110px" }
def CRM(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  ks_score = ks(G)
  degs = np.array([deg for v, deg in G.degree])

  # computing the shell vectors
  A = nx.adjacency_matrix(G).astype(bool).toarray()
  SV = np.zeros(shape=(len(G.nodes), np.max(ks_score) + 1))

  for node in G.nodes:
    for core in range(np.max(ks_score) + 1):
      SV[node, core] = np.sum((ks_score == core) & A[node])
  
  SCC = np.zeros_like(G.nodes)
  for node in G.nodes:
    scc_temp = 0
    for neighb in G.neighbors(node):
      scc_temp += (2 - np.corrcoef(SV[node], SV[neighb])[0, 1]) + (2* (degs[node]/np.max(degs)) + 1)
    SCC[node] = scc_temp
  
  CRM_score = np.zeros_like(G.nodes)
  for node in G.nodes:
    crm_temp = 0
    for neighb in G.neighbors(node):
      crm_temp += SCC[neighb]
    CRM_score[node] = crm_temp
  
  return CRM_score



In [24]:
#@title ECRM { form-width: "110px" }
def ECRM(G_in):

  G = nx.Graph()
  G.add_nodes_from(np.array(G_in.nodes))
  G.add_edges_from(np.array(G_in.edges))

  CRM_score = CRM(G)

  ECRM_score = np.zeros_like(G.nodes)
  for node in G.nodes:
    ecrm_temp = 0
    for neighb in G.neighbors(node):
      ecrm_temp += CRM_score[neighb]
    ECRM_score[node] = ecrm_temp
  
  return ECRM_score

In [25]:
#@title CN { form-width: "110px" }
def CN(G):

  ks_arr = ks(G)
  _, eq_rank = ks_if_2(G)

  # print(ks_arr[:10])
  # print(eq_rank[:10])

  cn = np.zeros_like(list(G.nodes), dtype=np.float32)
  for u in G.nodes():
    for v in G.neighbors(u):
      if ks_arr[u] > ks_arr[v]: # lower neighbour
        cn[u] += 0.091
      elif ks_arr[u] < ks_arr[v]: # upper neighbour
        cn[u] += 0.364
      elif ks_arr[u] == ks_arr[v] and eq_rank[u] > eq_rank[v]: # eq_lower
        cn[u] += 0.227
      else:
        cn[u] += 0.318

  return cn


In [26]:
G9.degree(1)

30

In [27]:
#@title H_index { form-width: "110px" }
def H_index(G):

  h_index = np.zeros_like(G.nodes)
  for u in G.nodes:
    sorted_neighbor_degrees = sorted((G.degree(v) for v in G.neighbors(u)), reverse=True)
    h = 0
    for i in range(1, len(sorted_neighbor_degrees)+1):
        if sorted_neighbor_degrees[i-1] < i:
            break
        h = i
    h_index[u] = h
  return h_index

In [28]:
#@title LH { form-width: "110px" }
def LH(G):

  H = H_index(G)

  LH = np.zeros_like(G.nodes)

  for u in G.nodes:
    LH[u] += H[u]
    for v in G.neighbors(u):
      LH[u] += H[v]
  
  return LH

In [29]:
#@title LS { form-width: "110px" }
def LS(G):

  ks_score = ks(G)

  ks_2 = np.zeros_like(G.nodes, dtype=np.float32)
  degs = [G.degree(u) for u in G.nodes]

  LS_score = np.zeros_like(G.nodes, dtype=np.float32)

  for u in G.nodes:
    LS_sum = 0
    N_u = set(G.neighbors(u))
    LS_u = 0
    for v in G.neighbors(u):
      N_v = set(G.neighbors(v))
      common = len(N_u & N_v)
      union = len(N_u | N_v)
      LS_u += 1 - (common/union)
    ks_2[u] = LS_u * ks_score[u] / degs[u]
  
  for u in G.nodes:
    N_u = set(G.neighbors(u))
    for v in G.neighbors(u):
      N_v = set(G.neighbors(v))
      common = len(N_u & N_v)
      union = len(N_u | N_v)
      LS_uv = 1 - (common/union)
      LS_score[u] += LS_uv * ks_2[v]
  
  return LS_score

In [30]:
#@title DSM { form-width: "110px" }

from numpy import linalg as LA
def DSM(G, beta):
  A = np.array(nx.to_numpy_matrix(G))
  A2 = LA.matrix_power(A, 2)
  A3 = LA.matrix_power(A, 3)
  A4 = LA.matrix_power(A, 4)
  # A5 = LA.matrix_power(A, 5)

  R = A*beta + A2*(beta**2) + A3*(beta**3) + A4*(beta**4) #+ A5*(beta**5)

  DSM_score = np.zeros_like(G.nodes)
  for v in G.nodes:
    DSM_score[v] = np.sum(R[:, v])
  
  return DSM_score

## Sampling Method

In [31]:
#@title optimalK { form-width: "150px" }
def optimalK(data, nrefs=6, maxClusters=18):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal
    

In [32]:
#@title elbow_optimal { form-width: "150px" }
def elbow_optimal(data, metric, k = 30):
  model = KElbowVisualizer(KMeans(), k=k, metric=metric)
  model.fit(data)
  return model.elbow_value_

In [33]:
#@title get_optimal_cluster_number{ form-width: "150px" }
def get_optimal_cluster_number(B, size_method='elbow', elbow_metric = 'silhouette'):

  if size_method == 'gap':
    optimal_clus_number = optimalK(B, nrefs=6, maxClusters=12)[0]
  elif size_method == 'elbow':
    optimal_clus_number = elbow_optimal(B, elbow_metric)
  else:
    print('Invalid size_method')
    raise Exception
  
  return optimal_clus_number

In [34]:
#@title get_manual_cluster_number { form-width: "150px" }
def get_manual_cluster_number(n_nodes, sample_size):
  n_samples =  int(np.ceil(sample_size * n_nodes))

  if n_samples < 10:
    return int(n_samples / 3)

  elif n_samples > 10 and n_samples < 30:
    return int(n_samples / 4)
  
  elif n_samples > 30 and n_samples < 70:
    return int(n_samples / 6)
  
  elif n_samples >70 and n_samples < 100:
    return int(n_samples / 8)
  
  elif n_samples > 100 and n_samples < 150:
    return int(n_samples / 10)
  
  elif n_samples > 150  and n_samples <200:
    return int(n_samples / 15)
  
  elif n_samples > 200:
    return int(n_samples / 20)
  


In [35]:
#@title get_cluster_sampling { form-width: "150px" }
def get_cluster_sampling(G, B, mask, samp_size, optimal_clus_number, size_method = 'elbow', elbow_metric = 'silhouette'):
  '''
  size_method: can be either 'elbow' or 'gap'
  elbow_metric: can be either 'distortion', 'silhouette', 'calinski_harabasz'
  '''

  n_samples = int(np.ceil(samp_size * len(G.nodes)))
  # B = get_features(G)

  # B = B[mask]

  # if size_method == 'gap':
  #   optimal_clus_number = optimalK(B, nrefs=6, maxClusters=12)[0]
  # elif size_method == 'elbow':
  #   optimal_clus_number = elbow_optimal(B, elbow_metric)
  # else:
  #   print('Invalid size_method')
  #   raise Exception

  sample_per_cluster = int(np.ceil(n_samples / optimal_clus_number))
  # out_of_bag = n_samples - sample_per_cluster * optimal_clus_number

  # if out_of_bag >= optimal_clus_number:
  #   sample_per_cluster += 1
  #   out_of_bag -= optimal_clus_number

  kmeans_model = KMeans(n_clusters=optimal_clus_number).fit(B)
  B_labels = kmeans_model.labels_
  B_indices = np.array(G.nodes())[mask]

  labels = sorted(set(B_labels))

  elem_per_clus = [len(B_indices[B_labels == label]) for label in labels]
  sample_per_clus = [sample_per_cluster if elem >= sample_per_cluster else elem \
                     for elem in elem_per_clus]
  out_of_bag = n_samples - np.sum(sample_per_clus)

  if out_of_bag > 0:

    cur = 0
    len_cluss = len(sample_per_clus)

    # while out_of_bag >= 1:
    #   if elem_per_clus[cur] > sample_per_clus[cur]:
    #     sample_per_clus[cur] += 1
    #     out_of_bag -= 1

    #   cur += 1
    #   cur = cur % len_cluss
 
    while out_of_bag >= 1:
      if sample_per_clus[cur] < sample_per_cluster:
        sample_per_clus[cur] += 1
        out_of_bag -= 1

      cur += 1
      cur = cur % len_cluss
    
    # print(sample_per_clus)


  sampled_nodes = []

  for label in labels:
    clust = list(B_indices[B_labels == label])
    sel_indices = list(random.sample(clust, sample_per_clus[label]))
    sampled_nodes.extend(sel_indices)

  sampled_nodes = sorted(sampled_nodes)

  # make sure that the number of output is equal to samp_size
  # print(len(sampled_nodes))
  # print(n_samples)
  # print(out_of_bag)
  assert out_of_bag <= 0

  # print('processing at cluster finished')

  # if len(sampled_nodes) != n_samples:
  #   print(len(sampled_nodes))
  #   print(n_samples)
  #   raise Exception

  return sampled_nodes

## ML Model

In [36]:
#@title Creating Features { form-width: "150px" }
def get_features(G, ks_coef = 10):

  # ks_score = ks(G)
  # ks_score = iks(G)
  ks_score = ks_if(G)

  A = np.array(nx.to_numpy_matrix(G))
  B = np.zeros_like(A)
  degs = np.array([deg for v, deg in G.degree])

  A1 = nx.adjacency_matrix(G).astype(bool).toarray()
  entropyy = np.zeros_like(G.nodes, dtype=np.float32)

  # for node in G.nodes:
  #   entropy_temp = 0

  #   for core_level in range(np.max(ks_score) + 1):
  #     if degs[node] > 0:
  #       p_i = np.sum((ks_score == core_level) & A1[node]) / degs[node]
  #       if p_i > 0 :
  #         entropy_temp +=  p_i * np.log(p_i)
    
  #   entropyy[node] = -entropy_temp


  alpha = 3
  gamma = 1  

  new_ks_score = ks_coef * ks_score
  new_deg = alpha * degs 

  # agg_score = new_deg * new_ks_score

  agg_score = alpha * degs + ks_coef * ks_score #+ gamma * entropyy

  for i in range(A.shape[0]):
    B[i] = A[i] * agg_score #* degs[i]

  return B

In [37]:
#@title ml_model { form-width: "150px" }
def ml_model(G, B, gt_bool, name, g, gt_inf, gt_indi, model, ks_coef, sample_size, optimal_cluster_number, config = {}, extended = True, sampling_method = 'cluster'):
  '''
  model can be: dt, rf, ada, GB, SVR, and Stacking
  gt can be:  'selected', or 'whole'; selection for the set of passed gt.
  '''
  try:
    assert len(gt_inf) == len(gt_indi)
  except:
    print(f'Graph {g}: {name}')
    print(f'len gt_inf: {len(gt_inf)}, len gt_indi:{len(gt_indi)}')
    raise Exception
                            
  # gt_bool = [True if v in gt_indi else False for v in G.nodes]

  if sampling_method == 'cluster':
    train_indices = get_cluster_sampling(G, B, gt_bool, sample_size, optimal_cluster_number)
    # print('returned from cluster sampling')
    # print(train_indices)

  else:
    n_samples = int(np.ceil(sample_size * len(G.nodes)))
    train_indices = np.array(sorted(set(random.sample(sorted(gt_indi), n_samples))))

    # try:
    #   train_indices = np.array(sorted(random.sample(list(gt_indi), n_samples)))
    # except Exception as e:
    #   print(gt_indi)
    #   print(gt_indi.shape)
    #   print(len(gt_indi))
    #   print(e)
    #   # print(type(gt_indi))
    #   raise Exception

  train_bool = np.array([True if v in train_indices else False for v in gt_indi])
  test_bool = ~ train_bool
  # print(np.array(gt_indi).shape)
  # print(np.array(test_bool).shape)
  test_indices = np.array(gt_indi)[test_bool]

  X = get_features(G, ks_coef)
  Y = gt_inf

  # print(len(train_indices))
  # print(sum(train_bool))
  x_train = [X[i] for i in train_indices]
  y_train = gt_inf[train_bool]

  x_test = [X[i] for i in test_indices]
  y_test = gt_inf[test_bool]

  all_nodes = np.array(G.nodes())
  # assert G.nodes[gt_bool][train_bool] == train_indices
  # print(all_nodes[gt_bool][train_bool])
  # print(train_indices)
  # print(gt_indi)
  # print(all_nodes[gt_bool][test_bool])
  # print(test_indices)

  # if gt_sel == 'whole':
  #   node_prob = np.random.rand(len(gt_inf))
  #   sample = np.array([True if p <= sample_size else False for p in node_prob])
  #   test = np.invert(sample)

    
  #   x_sample = X[sample]
  #   x_test = X[test]


  #   Y = gt_inf
  #   y_sample = Y[sample]
  #   y_test = Y[test]
  # else: # gt_sel == 'selected'
  #   node_prob = np.random.rand(len(G.nodes))
  #   sample = np.array([True if p <= sample_size else False for p in node_prob])

  #   X = get_features(G, beta)
  #   x_sample = X[sample]
  #   x_test = X[gt_tip_idx]

  #   y_sample_inds = np.arange(len(node_prob))[sample]
  #   y_sample = np.zeros_like(x_sample)
  #   for idx, samp_indx in enumerate(y_sample_inds):
  #     y_sample[idx] = get_TIP_inf(G, samp_indx, thres_tip)
    
  #   y_test = gt_tip



  # decision tree regressor
  if model == 'dt':
    regressor = DecisionTreeRegressor()
  elif model == 'rf':
    regressor = RandomForestRegressor()
  elif model == 'ada':
    regressor = AdaBoostRegressor()
  elif model == 'GB':
    regressor = GradientBoostingRegressor()
  elif model == 'SVR':
    if len(config) > 0:
      regressor = SVR(**config)
    elif len(config) == 0:
      regressor = SVR()
  elif model == 'KNN':
    if len(config) > 0:
      regressor = KNeighborsRegressor(**config)
    else:
      regressor = KNeighborsRegressor()
  elif model == 'PLS_reg':
    if len(config) > 0:
      regressor = PLSRegression(**config)
    else:
      regressor = PLSRegression()
  elif model == 'PLS_cano':
    if len(config) > 0:
      regressor = PLSCanonical(**config)
    else:
      regressor = PLSCanonical()
  elif model == 'GP_reg':
    if len(config) > 0:
      regressor = GaussianProcessRegressor(**config)
    else:
      regressor = GaussianProcessRegressor()
  elif model == 'MLP_reg':
    if len(config) > 0:
      regressor = MLPRegressor(**config)
    else:
      regressor = MLPRegressor()
  elif model == 'Stacking':
    # final_estimator = GradientBoostingRegressor(n_estimators=25, subsample=0.5, 
    #                                             min_samples_leaf=25, max_features=1, random_state=42)
    final_estimator = SVR()
    regressor = StackingRegressor(estimators=config, final_estimator=final_estimator)
  elif model == 'Voting':
    regressor = VotingRegressor(estimators=config)
  elif model == 'Bagging':
    regressor = BaggingRegressor(**config)
  
 
  # print('Start training')
  regressor.fit(x_train, y_train)
  # print('Training finished')

  if extended:
    score_raw_all = regressor.predict(X)
    score_test = score_raw_all[gt_bool][test_bool]
    # score_extended = np.zeros_like(score_raw_test)
    # ks_score_all = ks(G)
    ks_score_all = ks_if(G)
    # ks_score_all = iks(G)

    cnc_score_all = cnc(G)

    degs = np.array([deg for v, deg in G.degree])
    degs = degs / np.max(degs)

    for idx, u in enumerate(test_indices):
      # score_extended[idx] += score_raw_test[idx]
      for v in G.neighbors(u):
        score_test[idx] += 0.25 * ks_score_all[v] * score_raw_all[v] #+ 0.1 * cnc_score_all[v] * score_raw_all[v] #* (2 * degs[v])

    score_final = score_test
    # print('Through extended!')
  else:
    score_final = regressor.predict(x_test)
    # print('Through ordinary !')

  return score_final, test_bool

# Getting Results

## Get results

In [38]:
#@title single result for a config { form-width: "150px" }
def result_for_config(g, gt, model, samp_size, ks_coef, extended, config, monotinicity = False):
  network_df = get_network_df()
  
  name, G, SIR_gt, SIR_indi, TIP_gt, TIP_indi = network_df.loc[g, ['name', 'graph', 'SIR_gt', 'SIR_indi', 'TIP_gt', 'TIP_indi']]

  # Getting the results of the model
  if gt == 'SIR':
    pred_inf, sel_test = ml_model(G, name, g, SIR_gt, SIR_indi,  model,  ks_coef, samp_size, config)
    pred_ranking = get_ranking(pred_inf)
  else: ## gt == 'TIP'
    pred_inf, sel_test = ml_model(G, name, g, TIP_gt, TIP_indi,  model,  ks_coef, samp_size, config)
    pred_ranking = get_ranking(pred_inf)

  if monotinicity:
    res = get_monotinicity_relation(pred_ranking)
  else:
    if gt == 'SIR':
      gt_ranking = get_ranking(SIR_gt[sel_test])
    else: ## gt == 'TIP'
      gt_ranking = get_ranking(TIP_gt[sel_test])

    tau_pred, p_value_pred = stats.kendalltau(gt_ranking, pred_ranking)
    res = tau_pred
  # print(tau_pred)
 
  return res

In [39]:
#@title get_shortcut_result { form-width: "150px" }
def get_shortcut_result(G, X, mask, name, g, SIR_gt, SIR_indi, TIP_gt, TIP_indi, gt, model, samp_size, optimal_cluster_number, ks_coef, extended, config, monotinicity = False, samp_method= 'cluster'):

  if gt == 'SIR':
    pred_inf, sel_test = ml_model(G, X, mask, name, g, SIR_gt, SIR_indi,  model,  ks_coef, samp_size, optimal_cluster_number, config, sampling_method=samp_method)
    pred_ranking = get_ranking(pred_inf)
  else: ## gt == 'TIP'
    pred_inf, sel_test = ml_model(G, X, mask, name, g, TIP_gt, TIP_indi,  model,  ks_coef, samp_size, optimal_cluster_number, config, sampling_method = samp_method)
    pred_ranking = get_ranking(pred_inf)

  if monotinicity:
    res = get_monotinicity_relation(pred_ranking)
  else:
    if gt == 'SIR':
      gt_ranking = get_ranking(SIR_gt[sel_test])
    else: ## gt == 'TIP'
      gt_ranking = get_ranking(TIP_gt[sel_test])

    tau_pred, p_value_pred = stats.kendalltau(gt_ranking, pred_ranking)
    res = tau_pred
  # print(tau_pred)
 
  return res

## Get Aggregated Results

In [56]:
#@title get_agg_results { form-width: "150px" }
def get_agg_results(gs, gt, samp_size = 0.005, run_number = 1, mono = False, iteration = 25, sampling_method = 'cluster', runtime = False):
  '''
  For getting results of 1 and 3.
  1: Corr results for the ml model = Mono: False
  3: Mono results for the ml model = Mono: True, iteration : 1
  gt : Can be euqal to SIR or TIP
  '''
  if runtime:
    iteartion = 1


  network_df = get_network_df()
  
  config={'kernel': 'rbf', 'gamma': 'scale', 'tol': 1e-05, 'cache_size': 1000, 'C': 2, 'epsilon': 0.1}
  extended = True
  ks_coef = 3
  model = 'SVR'
  # samp_size = 0.005

  if gt == 'TIP':
    iteration = 1

  corr_pred = []
  run_times = []

  for g in gs:
    res_cur = []
    time_cur = []

    name, G, SIR_gt, SIR_indi, size_sir, TIP_gt, TIP_indi, size_tip = network_df.loc[g, ['name', 'graph', 'SIR_gt', 'SIR_indi', 'size_sir', 'SIR_gt', 'SIR_indi', 'size_sir']]
    B = get_features(G)
    # print(SIR_indi)
    # print(len(SIR_indi))
    # raise Exception
    
    # Computing a mask for the elements that we have in ground truth
    if gt == 'SIR' and size_sir < 1:
      mask= [True if v in SIR_indi else False for v in G.nodes]
    elif gt == 'TIP' and size_tip < 1:
      mask= [True if v in TIP_indi else False for v in G.nodes]
    else:
      mask = np.ones_like(G.nodes(), dtype=bool)

    B = B[mask]

    # Getting the optimal cluster number if the method is cluster
    if sampling_method == 'cluster':
      # opt_clus_number = get_optimal_cluster_number(B)
      # opt_clus_number = get_manual_cluster_number(len(G.nodes), samp_size)
      opt_clus_number = 2
      print(f'Optimal cluster number is : {opt_clus_number}')
    else:
      opt_clus_number = -1
    
    
    for i in range(iteration):
      time1 = time.time_ns()
      result_run = get_shortcut_result(G, B, mask, name, g, SIR_gt, sorted(set(SIR_indi)), TIP_gt, sorted(set(TIP_indi)), gt, model, samp_size, opt_clus_number, ks_coef, extended, config, mono, sampling_method)
      time2 = time.time_ns()
      # print(result_run)
      time_cur.append(time2-time1)
      res_cur.append(result_run)
    # print(f'results for {g}: {np.mean(res_cur)}')
    print(f'runtimes for {g}: {np.mean(time_cur)}')
    corr_pred.append(np.mean(res_cur))
    run_times.append(np.mean(time_cur))

  if runtime:
    pd.DataFrame(run_times, index=gs, columns=['run time']).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/7_run-times_{gs}.csv')

  else:
    if mono:
      pd.DataFrame(corr_pred, index=gs, columns=['corr']).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/3_mono_{gs}_{sampling_method}_{run_number}_{samp_size}.csv')

    else:
      pd.DataFrame(corr_pred, index=gs, columns=['corr']).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/1_corr_{gs}_{sampling_method}_{run_number}_{samp_size}.csv')
  
  

  print(f'Finished. You can now see the results.')  


In [58]:
get_agg_results(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G12', 'G8'], gt='SIR', iteration = 20, sampling_method = 'none', mono = False, runtime=True)

runtimes for G5: 169724300.85
runtimes for G9: 370396516.35
runtimes for G6: 1073395074.85
runtimes for G10: 615621351.7
runtimes for G11: 1238246681.15
runtimes for G7: 1131281183.85
runtimes for G12: 3128009318.55
runtimes for G8: 17844927586.75
Finished. You can now see the results.


In [47]:
# get_agg_results(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G8'], gt='SIR', iteration = 20, sampling_method = 'none', mono = False, runtime=True)

runtimes for G5: 174210323.45
runtimes for G9: 370329878.1
runtimes for G6: 1102855146.45
runtimes for G10: 636761930.9
runtimes for G11: 1270607788.6
runtimes for G7: 1137955900.8
runtimes for G8: 18164127997.1
Finished. You can now see the results.


In [ ]:
# get_agg_results(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G8'], gt='SIR', iteration = 20, sampling_method = 'none', mono = False)

In [ ]:
# get_agg_results([ 'G12'], gt='SIR', run_number=2, iteration = 20, sampling_method = 'cluster', mono = False, samp_size=0.005)

In [53]:
#@title get_agg_results_others { form-width: "150px" }
def get_agg_results_others(gs, run_number = 1, mono='corr', runtime = False):

  '''
  Results of 2, change of corr with dynamics parameters for other models (baselines).
  g: can be either of 'G1' to 'G8'
  '''

  network_df = get_network_df() 
  # methods =      ['ks' , 'mcde', 'CN', 'DSM', 'mmd', 'H_index', 'LS']
  # methods_name = ['KS', 'MCDE', 'CN', 'DS', 'MMD'  , 'H-Index', 'LS']

  # methods =      ['ks' , 'mcde', 'CN', 'DSM', 'mmd', 'H_index', 'DSR', 'LS', 'CRM']
  # methods_name = ['KS', 'MCDE', 'CN', 'DS', 'MMD'  , 'H-Index', 'DSR', 'LS', 'ERM']

  methods =      ['ks' , 'mcde', 'CN', 'ks_if', 'DSM', 'mmd', 'H_index', 'LS']
  methods_name = ['KS', 'MCDE', 'CN',  'KS-IF', 'DS', 'MMD'  , 'H-Index', 'LS']

  # methods =      ['DSR', 'CRM']
  # methods_name = ['DSR', 'ERM']

  agg_results = []
  g_names = []
  run_times = []

  for g in gs:
    name, G, gt_inf, beta_th= network_df.loc[g, ['name', 'graph', 'SIR_gt', 'thresh_sir']]
    gt_ranking = get_ranking(gt_inf)
    g_names.append(name)
    # ks, mmd, cnc+, ks-if, mcde, dsr, edsr, crm, ecrm
    # methods = ['ks', 'mmd', 'cnc', 'ks_if', 'mcde', 'DSR', 'EDSR', 'CRM', 'ECRM']
    # methods_name = ['KS', 'MMD', 'CNC+', 'KS-IF', 'MCDE', 'DSR', 'EDSR', 'CRM', 'ECRM']

    methods_rankings = {}
    G_cur_res = []
    runtimes_cur = []
    for method in methods:
      time1 = time.time_ns()
      if method == 'DSM':
        scores = DSM(G, beta_th)
      else:
        scores = eval(method)(G)
      method_ranking = get_ranking(scores)
      time2 = time.time_ns()

      if runtime:
        runtimes_cur.append(time2-time1)

      else:
        if mono == 'mono':
          G_cur_res.append(get_monotinicity_relation(method_ranking))
        else:
          tau, p_value = stats.kendalltau(gt_ranking, method_ranking)
          G_cur_res.append(tau)
    
    agg_results.append(G_cur_res)
    run_times.append(runtimes_cur)

    print(f'process for {g}:{name} is over!')
  
  if runtime:
    pd.DataFrame(run_times, index=g_names, columns=methods_name).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/7_runtime_others_{gs}.csv')
  else:
    pd.DataFrame(agg_results, index=g_names, columns=methods_name).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/1_corr_others_{gs}_{mono}_{run_number}_.csv')

In [54]:
get_agg_results_others(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G12', 'G8'], run_number = 1, mono='mono', runtime=True)

process for G5:euroroad is over!
process for G9:email is over!
process for G6:hamster is over!
process for G10:cietseer is over!
process for G11:collab is over!
process for G7:powergrid is over!
process for G12:delaunay is over!
process for G8:pgp is over!


In [ ]:
# get_agg_results_others(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G12', 'G8'], run_number = 1, mono='mono')

In [ ]:
# get_agg_results_others(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G8'], run_number = 1)

In [ ]:
# get_agg_results_others(['G9','G11', 'G12'], run_number = 1)

In [ ]:
# get_agg_results_others(['G1', 'G2', 'G3', 'G4', 'G5','G6', 'G7', 'G8'], run_number = 4, mono='mono')

In [ ]:
#@title get_beta_sensi { form-width: "150px" }
def get_beta_sensi(gs, samp_size = 0.005, run_number = 1, samp_method = 'none'):
  '''
  Results of 2, change of corr with dynamics parameters for the ml model.
  '''

  network_df = get_network_df()
  
  config={'kernel': 'rbf', 'gamma': 'scale', 'tol': 1e-05, 'cache_size': 1000, 'C': 2, 'epsilon': 0.1}
  extended = True
  ks_coef = 3
  model = 'SVR'
  low_range = 0.15
  up_range = 0.15

  for g in gs:
    name, G, beta_th, range_beta, step= network_df.loc[g, ['name', 'graph', 'thresh_sir', 'range_sir', 'step_sir']]
    betas = [np.round(i, 4) for i in np.arange(beta_th - range_beta, beta_th + range_beta, step)]
    B = get_features(G)
    corrs = []

    print(f'For graph {g}:')
    for beta_inf in betas:
      try:
        gt_inf = pickle.load(open(f'/content/drive/My Drive/PhD_IM/datasets/gt_data/{g}_{name}/{g}_{beta_inf}_gt.pckl', "rb"))
      except:
        corrs.append(-1)
        continue
      mask = np.ones_like(G.nodes(), dtype=bool)
      # Getting the results of the model
      # pred_inf, sel_test = ml_model(G, gt_inf, model,  beta, extended, config, sample_size=samp_size)
      pred_inf, sel_test = ml_model(G, B, mask, name, g, gt_inf, list(G.nodes), model,  ks_coef, samp_size, 2, config, sampling_method=samp_method)

      gt_ranking = get_ranking(gt_inf[sel_test])
      # M_R_gt = get_monotinicity_relation(gt_ranking)

      pred_ranking = get_ranking(pred_inf)
      # M_R_pred = get_monotinicity_relation(pred_ranking)
      tau_pred, p_value_pred = stats.kendalltau(gt_ranking, pred_ranking)

      corrs.append(tau_pred)
      print(f'beta:{beta_inf}, corr:{tau_pred}')
    
    pd.DataFrame(corrs, index=betas, columns=[g]).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/2_corr_sensi_{g}_{samp_method}_{run_number}_{samp_size}.csv')
  

In [ ]:
# get_beta_sensi(['G10'], run_number=1, samp_size=0.005)

In [ ]:
#@title get_beta_sensi_others { form-width: "150px" }
def get_beta_sensi_others(g, run_number = 1):
  '''
  Results of 2, change of corr with dynamics parameters for other models (baselines).
  g: can be either of 'G1' to 'G8'
  '''

  network_df = get_network_df()
  name, G, beta_th, range_beta, step= network_df.loc[g, ['name', 'graph', 'thresh_sir', 'range_sir', 'step_sir']]

  # ks, mmd, cnc+, ks-if, mcde, dsr, edsr, crm, ecrm
  # methods = ['ks', 'mmd', 'cnc', 'ks_if', 'mcde', 'DSR', 'EDSR', 'CRM', 'ECRM']
  # methods_name = ['KS', 'MMD', 'CNC+', 'KS-IF', 'MCDE', 'DSR', 'EDSR', 'CRM', 'ECRM']

  # methods = ['cnc', 'mcde', 'ks_if', 'mmd', 'DSR', 'EDSR', 'CRM', 'ECRM']
  # methods_name = ['CNC+', 'MCDE', 'KS-IF', 'MMD', 'DSR', 'EDSR', 'CRM', 'ECRM']

  methods =      ['ks' , 'mcde', 'CN', 'ks_if', 'DSM', 'mmd', 'H_index', 'DSR', 'LS', 'CRM']
  methods_name = ['KS', 'MCDE', 'CN',  'KS-IF', 'DS', 'MMD'  , 'H-Index', 'DSR', 'LS', 'ERM']

  # methods = ['DSM']
  # methods_name = ['DSM']

  betas = [np.round(i, 4) for i in np.arange(beta_th - range_beta, beta_th + range_beta, step)]

  methods_rankings = {}

  for method in methods:
    if method == 'DSM':
      scores = DSM(G, beta_th)
    else:
      scores = eval(method)(G)

    method_ranking = get_ranking(scores)
    methods_rankings[method] = method_ranking

  agg_results = []

  for beta in betas:
    if beta >= 0:
      try:
        gt_inf = pickle.load(open(f'/content/drive/My Drive/PhD_IM/datasets/gt_data/{g}_{name}/{g}_{beta}_gt.pckl', "rb"))
        gt_ranking = get_ranking(gt_inf)

        results = []

        for method in methods:
          tau, p_value = stats.kendalltau(gt_ranking, methods_rankings[method])
          results.append(tau)

        agg_results.append(results)
        print(f'for beta: {beta} is done!')
      except:
        pass
    
  
  pd.DataFrame(agg_results, index=betas, columns=methods_name).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/2_corr_sensi_others_{g}_{run_number}_.csv')
  # print(f'Graph {name}, {g} is done')
  # return np.round(agg_results, 3)

In [ ]:
# get_beta_sensi_others('G10', run_number=1)

In [ ]:
#@title get_jacard_sim { form-width: "150px" }
def get_jacard_sim(gs, run_number = 1, samp_size = 0.005, samp_method = 'cluster'):
  '''
  Results of 4 for the ml model
  Computes the jacard similarity between the ranking of this method, and the ground-truth ranking.
  gs: List of the graphs by name
  '''
  network_df = get_network_df()

  config={'kernel': 'rbf', 'gamma': 'scale', 'tol': 1e-05, 'cache_size': 1000, 'C': 2, 'epsilon': 0.1}
  extended = True
  ks_coef = 3
  model = 'SVR'
  
  data = []
  JS_agg = []
  pres_agg = []

  rank_cut = np.linspace(10, 300, num = 15, dtype=np.int)
  df_JS = pd.DataFrame(index = rank_cut, columns=[gs])

  JS_agg = []

  for idx, g in enumerate(gs):
    name, G, gt_inf  = network_df.loc[g, ['name', 'graph', 'SIR_gt']]
    mask = np.ones_like(G.nodes(), dtype=bool)
    B = get_features(G)
    # pred_inf, sel_test = ml_model(G, gt_inf, model,  beta, extended, config, sample_size=samp_size)
    pred_inf, sel_test = ml_model(G, B, mask, name, g, gt_inf, list(G.nodes), model,  ks_coef, samp_size, 2, config, sampling_method=samp_method)
    pred_ranking = get_ranking(pred_inf)
    pred_sorted_ranking = get_sorted_ranking(pred_ranking)

    gt_ranking = get_ranking(gt_inf[sel_test])
    gt_sorted_ranking = get_sorted_ranking(gt_ranking)

    # rank_cut = [10 + int(i*10) for i in range(int(len(set(gt_ranking)) / 10))]
    # rank_cut = [10 + int(i*10) for i in np.linspace(10, int(len(set(gt_ranking)) / 5), num = 15)]
    # rank_cut = np.linspace(10, 300, num = 15, dtype=np.int)
    
    JS_cur = []
    pres_cur = []

    for rank in rank_cut:
      gt_rank = gt_sorted_ranking[:rank]
      pred_rank = pred_sorted_ranking[:rank]
      num_indentified = len(set(gt_rank).intersection(set(pred_rank)))
      JS = num_indentified / len(set(gt_rank).union(set(pred_rank)))
      JS_cur.append(JS)

      pres = num_indentified/ len(gt_rank)
      pres_cur.append(pres)

    # df_JS[g] = JS_cur
    JS_agg.append(JS_cur)
  
  JS_agg = np.array(JS_agg).T
  pd.DataFrame(JS_agg, index = rank_cut, columns=[gs]).to_csv(f'/content/drive/My Drive/PhD_IM/Project 2- IM via Machine Learning/2_results/4_Jaccard_ml_{samp_method}_{run_number}_{gs}_{samp_size}.csv')


In [ ]:
# get_jacard_sim(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G12', 'G8'], run_number=1, samp_method = 'none')

In [ ]:
#@title get_jacard_sim_others { form-width: "150px" }
def get_jacard_sim_others(g, run_number = 1):
  '''
  Results of 4 for other models.
  '''

  network_df = get_network_df()

  # methods = ['ks', 'mmd', 'cnc', 'ks_if', 'mcde', 'DSR', 'EDSR', 'CRM', 'ECRM']
  # methods_name = ['KS', 'MMD', 'CNC+', 'KS-IF', 'MCDE', 'DSR', 'EDSR', 'CRM', 'ECRM']

  # methods = ['cnc', 'mcde', 'ks_if', 'mmd', 'DSR', 'EDSR', 'CRM', 'ECRM']
  # methods_name = ['CNC+', 'MCDE', 'KS-IF', 'MMD', 'DSR', 'EDSR', 'CRM', 'ECRM']

  methods =      ['ks' , 'mcde', 'CN', 'ks_if', 'DSM', 'mmd', 'H_index', 'DSR', 'LS', 'CRM']
  methods_name = ['KS', 'MCDE', 'CN',  'KS-IF', 'DS', 'MMD'  , 'H-Index', 'DSR', 'LS', 'ERM']

  name, G, gt_inf, beta  = network_df.loc[g, ['name', 'graph', 'SIR_gt', 'thresh_sir']]
  gt_ranking = get_ranking(gt_inf)
  gt_sorted_ranking = get_sorted_ranking(gt_ranking)
  rank_cut = np.linspace(10, 300, num = 15, dtype=np.int)

  methods_sorted_rankings = {}

  for method in methods:

    if method == 'DSM':
      scores = DSM(G, beta)
    else:
      scores = eval(method)(G)

    method_ranking = get_ranking(scores)
    method_sorted_ranking = get_sorted_ranking(method_ranking)
    methods_sorted_rankings[method] = method_sorted_ranking

  JS_agg = []

  for rank in rank_cut:
    # gt_inf = pickle.load(open(f'/content/drive/My Drive/PhD_IM/datasets/gt_data/{g}_{name}/{g}_{beta}_gt.pckl', "rb"))
    JS_cur = []
    for method in methods:
      gt_rank = gt_sorted_ranking[:rank]
      method_rank = methods_sorted_rankings[method][:rank]
      num_indentified = len(set(gt_rank).intersection(set(method_rank)))
      JS = num_indentified / len(set(gt_rank).union(set(method_rank)))
      JS_cur.append(JS)

    JS_agg.append(JS_cur)
  
  pd.DataFrame(JS_agg, index=rank_cut, columns=methods_name).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/4_Jaccard_sim_others_{run_number}_{g}.csv')

In [ ]:
# get_jacard_sim_others('G9', run_number = 2)

In [ ]:
#@title get_rank_dist { form-width: "150px" }
def get_rank_dist(gs, run_number = 1, steps = 40, samp_size = 0.005, sampling_method = 'none'):
  '''
  Results of 5 for the ml model.
  '''

  network_df = get_network_df()

  config={'kernel': 'rbf', 'gamma': 'scale', 'tol': 1e-05, 'cache_size': 1000, 'C': 2, 'epsilon': 0.1}
  extended = True
  ks_coef = 3
  model = 'SVR'

  dists = []

  for idx, g in enumerate(gs):
    name, G, gt_inf = network_df.loc[g, ['name', 'graph', 'SIR_gt']]
    
    mask = np.ones_like(G.nodes(), dtype=bool)
    B = get_features(G)
    # pred_inf, sel_test = ml_model(G, gt_inf, model,  ks_coef, extended, config, sample_size=samp_size)
    pred_inf, sel_test = ml_model(G, B, mask, name, g, gt_inf, list(G.nodes), model,  ks_coef, samp_size, 2, config, sampling_method='none')
    pred_ranking = get_ranking(pred_inf)

    ranks = np.linspace(1, len(G.nodes), num =steps)

    prev = 1
    dist_cur = []

    for rank in ranks:
      remain = len(pred_ranking[pred_ranking > rank]) / len(pred_ranking)
      dist_cur.append(np.round(prev - remain, 3))
      prev = remain
    dists.append(dist_cur)
  
  dists = np.array(dists).T
  
  pd.DataFrame(dists, columns=gs).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/5_rank_dist_ml_{run_number}_{gs}.csv')

In [ ]:
# get_rank_dist(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G12', 'G8'], run_number = 1)

In [ ]:
#@title get_rank_dist_others { form-width: "150px" }
def get_rank_dist_others(gs, run_number = 1, steps = 40):
  '''
  Results of 5 for other models.
  '''

  network_df = get_network_df()

  # methods = ['ks', 'mmd', 'cnc', 'ks_if', 'mcde', 'DSR', 'EDSR', 'CRM', 'ECRM']
  # methods_name = ['KS', 'MMD', 'CNC+', 'KS-IF', 'MCDE', 'DSR', 'EDSR', 'CRM', 'ECRM']

  methods =      ['ks' , 'mcde', 'CN', 'ks_if', 'DSM', 'mmd', 'H_index', 'DSR', 'LS', 'CRM']
  methods_name = ['KS', 'MCDE', 'CN',  'KS-IF', 'DS', 'MMD'  , 'H-Index', 'DSR', 'LS', 'ERM']

  # methods = ['DSM']
  # methods_name = ['DSM']
  

  for idx, g in enumerate(gs):
    name, G, gt_inf, beta = network_df.loc[g, ['name', 'graph', 'SIR_gt', 'thresh_sir']]

    ranks = np.linspace(1, len(G.nodes), num =steps, dtype=np.int)

    methods_ranking = {}
    dists = []

    for method in methods:
      if method == 'DSM':
        scores = DSM(G, beta)
      else:
        scores = eval(method)(G)
        
      method_ranking = get_ranking(scores)
      methods_ranking[method] = method_ranking

    prevs = np.ones_like(methods, dtype=np.float) 

    for rank in ranks:
      dist_cur = []
      for idx, method in enumerate(methods):
        ranking_cur = methods_ranking[method]
        remain = len(ranking_cur[ranking_cur > rank]) / len(ranking_cur)
        dist_cur.append(np.round(prevs[idx] - remain, 3))
        prevs[idx] = remain
      dists.append(dist_cur)
  
    pd.DataFrame(dists, columns=methods_name, index=ranks).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/5_rank_dist_others_{g}_{run_number}.csv')
    print(f'graph {g} is finished !!')

In [ ]:
# get_rank_dist_others(['G6', 'G10'])

In [ ]:
# get_rank_dist_others(['G5', 'G7', 'G8'], run_number = 2)

In [ ]:
#@title get_other_ml_models { form-width: "150px" }
def get_other_ml_models(gs, run_number = 1, samp_size = 0.005, mono = False):


  network_df = get_network_df()
  others = ['MLP_reg', 'KNN',  'dt', 'rf', 'ada', 'GB', 'SVR', 'Stacking', 'Voting']
  # others = ['GP_reg']
  # others = ['Stacking', 'Voting']

  configs = {}
  configs['SVR']={'kernel': 'rbf', 'gamma': 'scale', 'tol': 1e-05, 'cache_size': 1000, 'C': 2, 'epsilon': 0.1}
  configs['Bagging']= {'base_estimator': SVR(C=2, cache_size=1000, coef0=0.0, degree=3, epsilon=0.1, gamma='scale', kernel='rbf', max_iter=-1, shrinking=True, tol=1e-05, verbose=False), 'n_estimators': 30, 'bootstrap': True, 'bootstrap_features': False, 'oob_score': False, 'warm_start': True, 'n_jobs': None}
  configs['MLP_reg'] = { 'activation': 'relu', 'solver': 'lbfgs', 'learning_rate': 'adaptive', 'alpha': 0.001, 'learning_rate_init': 0.0001, 'max_iter': 600, 'tol': 0.0001, 'warm_start': True, 'early_stopping': True, 'beta_1': 0.99, 'beta_2': 0.999, 'epsilon': 1e-08, 'n_iter_no_change': 10, 'max_fun': 15000}
  # configs['KNN'] = {'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 50, 'p': 1, 'metric': 'minkowski', 'n_jobs': None}

  corrs = []
  # print(f'For other ml methods {others}')
  for g in gs:
    name, G, gt_inf = network_df.loc[g, ['name', 'graph', 'SIR_gt']]
    mask = np.ones_like(G.nodes(), dtype=bool)
    B = get_features(G)
    
    if g == 'G8':
      hidden_layer_size = tuple(sorted(np.linspace(10, len(G.nodes) / 5, num = 3, dtype=np.int), reverse=True))
    else:
      hidden_layer_size = tuple(sorted(np.linspace(10, len(G.nodes) / 5, num = 3, dtype=np.int), reverse=True))
    
    configs['MLP_reg']['hidden_layer_sizes'] = hidden_layer_size

    corrs_cur = []

    # print(f'For {g}')
    for method in others:

      if method in configs.keys():
        # pred_inf, sel_test = ml_model(G, gt_inf, method, 3, True, configs[method], sample_size=samp_size)
        pred_inf, sel_test = ml_model(G, B, mask, name, g, gt_inf, list(G.nodes), method,  3, samp_size, 2, configs[method], sampling_method='none')
      elif method in ['Voting', 'Stacking']:
        estimators = [('svr', SVR(**configs['SVR'])), ('ada', AdaBoostRegressor()), ('rf', RandomForestRegressor())]#('knn', KNeighborsRegressor())]#, ('rf', RandomForestRegressor()),]# ('knn', KNeighborsRegressor(**KNN_config))]
        # pred_inf, sel_test = ml_model(G, gt_inf, method, 3, True, estimators, sample_size=samp_size)
        pred_inf, sel_test = ml_model(G, B, mask, name, g, gt_inf, list(G.nodes), method,  3, samp_size, 2, estimators, sampling_method='none')
      else:
        try:
          # pred_inf, sel_test = ml_model(G, gt_inf, method, 3, True, sample_size=samp_size)
          pred_inf, sel_test = ml_model(G, B, mask, name, g, gt_inf, list(G.nodes), method,  3, samp_size, 2, sampling_method='none')
        except:
          print(g)
          print(method)
          raise(Exception)
      
      pred_ranking = get_ranking(pred_inf)
      gt_ranking = get_ranking(gt_inf[sel_test])

      if mono:
        res = get_monotinicity_relation(pred_ranking)

      else:
        res, p_value_pred = stats.kendalltau(gt_ranking, pred_ranking)

      corrs_cur.append(res)
      print(f'{method} is done!')
    print(f'Corrs for graph {g} is {corrs_cur}')

    corrs.append(corrs_cur)
  
  if mono:
    pd.DataFrame(corrs, index=gs, columns=others).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/6_mono_ml_others_{gs}_{samp_size}_{run_number}.csv')
  else:
    pd.DataFrame(corrs, index=gs, columns=others).to_csv(f'/content/drive/MyDrive/PhD_IM/Project 2- IM via Machine Learning/2_results/6_corr_ml_others_{gs}_{samp_size}_{run_number}.csv')
  
  

In [ ]:
# get_other_ml_models(['G5', 'G9', 'G6', 'G10', 'G11', 'G7', 'G12', 'G8'], run_number=1, mono = False)

# Plotting the X-space

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE


def plot_xspace(G):
  model = TSNE(learning_rate = 100)

  B = get_features(G)
  transformed = model.fit_transform(B)

  xs = transformed[:, 0]
  ys = transformed[:, 1]

  plt.plot(xs, ys)

  plt.show()

In [ ]:
# plot_xspace(G5)

In [ ]:
# plot_xspace(G6)

In [ ]:
# plot_xspace(G7)

In [ ]:
# plot_xspace(G8)

In [ ]:
# plot_xspace(G9)

In [ ]:
# plot_xspace(G10)

In [ ]:
# plot_xspace(G11)

In [ ]:
# plot_xspace(G12)